# Implement your own Decision Tree/Random Forest!


In this python notebook, you will create a basic decision tree on pandas data, and train a classifier on the Iris dataset. Then, you will implement a type of bagging and create a random forest classifier!

First, import the required modules:

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

Then import and preview the data:

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df['species'] = iris.target 
df.head()

,0,1,2,3,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


We have four features labeled 0, 1, 2, and 3. These stand for the length and the width of the sepals and petals, in centimeters. We want to use these four features to predict whether the species is one of three types of Iris plant, labeled 0, 1, or 2. 

Now, we split the dataset into training and test samples.

In [3]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
train = train.drop(['is_train'], axis = 1)
test = test.drop(['is_train'], axis = 1)

# Disorder (Splitting Metric)

First, we want to implement some measure of disorder in a set of data.

Implement either information gain or GINI impurity discussed in class. (for reference the equations are in 189 notes here https://www.eecs189.org/static/notes/n25.pdf) 


The argument `data` is a pandas dataframe containing the features and labels of several data points. We calculate disorder based on the labels, or the last column of the data. Note: make sure that you make this function work for different data (i.e. your function should work for data of different dimensions).

In [4]:
def disorder(data):
    labels = data.iloc[:, -1]
    unique_labels = {}
    for label in labels:
        if label in unique_labels:
            unique_labels[label] = unique_labels[label] + 1
        else:
            unique_labels[label] = 1
    total = len(labels)
    impurity = 0
    for key in unique_labels:
        impurity = impurity + (unique_labels[key]/total)**2
    return 1 - impurity

We now create a split function. This function takes in a dataset, and indices for a row and column. We then return two dataframes split on the `column`th feature. The left dataset should contain all of the data where the `column`th feature is greater or equal to the `column`th feature of the `row`th datapoint, and the right should contain the rest. 

In [5]:
def split_on_row_column(data, row, column):
    left = data[data.iloc[:, column] >= data.iloc[row, column]]
    right =  data[data.iloc[:, column] < data.iloc[row, column]]
    return left, right

We now want to define our recursive tree class. During training, there are two cases for a node. If the data is all one label, the node is a leaf node, and we return this value during inference. If the data is not all the same label, we find the best split of the data by iterating through all of features and rows in the data. Use the split function defined above to find the best split. Use the disorder metric you implemented in the function above.

Inference takes in a row of a pandas dataframes and returns the predicted class. 

In [6]:
class Node:
    def __init__(self, data):
        self.data = data
        self.result = -1
    
    def train(self):
        labels = self.data.iloc[:, -1]
        if len(pd.unique(labels)) == 1:
            self.result = labels.iloc[0]
        else:
            best_split = (disorder(self.data), 0, 0)
            for i in range(self.data.shape[0]):
                for j in range(self.data.shape[1]):
                    left_data, right_data = split_on_row_column(self.data, i, j)
                    gini = left_data.shape[0]/self.data.shape[0] * disorder(left_data) + right_data.shape[0]/self.data.shape[0] * disorder(right_data)
                    if gini < best_split[0] and left_data.shape[0] != 0 and right_data.shape[0] != 0:
                        best_split = (gini, i, j)
            left_data, right_data = split_on_row_column(self.data, best_split[1], best_split[2])
            self.split = self.data.iloc[best_split[1], best_split[2]]
            self.feature = best_split[2]
            self.left = Node(left_data)
            self.right = Node(right_data)
            self.left.train()
            self.right.train()
        
    def inference(self, x):
        if self.result == -1:
            if x.iloc[self.feature] >= self.split:
                return self.left.inference(x)
            else:
                return self.right.inference(x)
        return self.result

Now initialize and train a decision tree:

In [7]:
tree = Node(train)
tree.train()

Note that we don't check the training accuracy here (why?). We now want to validate our tree on the test dataset:

In [8]:
def validate(model, data):
    ct = 0
    corr = 0
    for i in range(test.shape[0]):
        data = test.iloc[i]
        ct += 1
        if model.inference(data) == data['species']:
            corr += 1
    return corr/ct

validate(tree, test)

1.0

## Random Forest!

Now we will implement data bagging with a random forest! The set up is similar to a single tree. We pass in the data to the forest, along with hyperparameters `n`, `frac`, and `m`, which correspond to the number of trees, the fraction of the dataset to use in each bag, the number or percentage of random features (depending on your own implementation) selected at each possible split. Note that the difference between random forests and just bagging  is that random forests select a random subset of features per bag while bagging assumes all features are present in each sample. A good estimate for m in a dataset with `num_features` is m = sqrt(`num_features`). In the inference step we tally the number of votes from each decision tree and return the label with the most amount of votes.

In [9]:
class Forest:
    def __init__(self, data, n, frac, m):
        self.data = data
        self.n = n
        self.frac = frac
        self.m = m
    
    def train(self):
        self.trees = []
        for i in range(self.n):
            data_select = self.data.sample(frac=self.frac)
            features = data_select.iloc[:, :-1].sample(n=self.m,axis='columns')
            features.append(data_select.iloc[:, -1])
            features.loc[:, len(features)] = data_select.iloc[:, -1]
            data_select = features
            tree = Node(data_select)
            tree.train()
            self.trees.append(tree)
    
    def inference(self, x):
        votes = {}
        for tree in self.trees:
            vote = tree.inference(x)
            if vote in votes:
                votes[vote] = votes[vote] + 1
            else:
                votes[vote] = 1
        return max(votes, key=votes.get)

Train and validate your forest!

In [10]:
forest = Forest(train, 30, .5, 3)
forest.train()

In [11]:
validate(forest, test)

0.8648648648648649